# Visualizing hypergraphs
As for pairwise networks, visualizing hypergraphs is surely a hard task and no algorithm can exaustively work for any given input structure. Here we show how to visualize some toy structures using the visualization function contained in the ```drawing``` module that heavily relies on [networkx](https://networkx.org/documentation/stable/reference/drawing.html) and [matplotlib](https://matplotlib.org/).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import xgi

Les us first create a small toy hypergraph containing edges of different sizes.

In [ ]:
H = xgi.Hypergraph()
H.add_edges_from(
    [[1, 2, 3], [3, 4, 5], [3, 6], [6, 7, 8, 9], [1, 4, 10, 11, 12], [1, 4]]
)

The first step for drawing a hypergraph is to choose a layout for the nodes.
At the moment the available layouts are:
* ```random_layout```: to position nodes uniformly at random in the unit square ([exactly as networkx](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.random_layout.html)).
* ```pairwise_spring_layout```: to position the nodes using the Fruchterman-Reingold force-directed algorithm on the projected graph. In this case the hypergraph is first projected into a graph (1-skeleton) using the ```xgi.convert_to_graph(H)``` function and then networkx's [spring_layout](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html) is applied. 
* ```barycenter_spring_layout```: to position the nodes using the Fruchterman-Reingold force-directed algorithm using an augmented version of the the graph projection of the hypergraph, where _phantom nodes_ (that we call barycenters) are created for each edge of order $d>1$ (composed by more than two nodes). Weights are then assigned to all hyperedges of order 1 (links) and to all connections to phantom nodes within each hyperedge to keep them together. Weights scale with the size of the hyperedges. Finally, the weighted version of networkx's [spring_layout](https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html) is applied.
* ```weighted_barycenter_spring_layout```: same as ```barycenter_spring_layout``, but here the weighted version of the Fruchterman-Reingold force-directed algorithm is used. Weights are assigned to all hyperedges of order 1 (links) and
    to all connections to phantom nodes within each hyperedge to keep them together. Weights scale with the order of the group interaction.
    
Each layout returns a dictionary that maps nodes ID into (x, y) coordinates.

In [ ]:
pos = xgi.barycenter_spring_layout(H)

We can now pass the ```pos``` dictionary to the ```drawing``` function:

In [ ]:
xgi.draw(H, pos)

**Colors of the hyperedges** match the hyperedge size by default, but any statistic can be used to color it as well. The default colormap can be changed by updating the default arguments. Both sequential and qualitative [colormaps](https://matplotlib.org/stable/tutorials/colors/colormaps.html) can be passed as an argument. Sequential colormaps would simply be discretized according to the sizes of the provided hypergraph:

In [ ]:
plt.figure(figsize=(10, 4))

# Sequential colormap
cmap = plt.cm.Paired

ax = plt.subplot(1, 2, 1)
xgi.draw(H, pos, ax=ax, edge_face_colormap=cmap)

# Qualitative colormap
cmap = plt.cm.Purples

ax = plt.subplot(1, 2, 2)
xgi.draw(H, pos, ax=ax, edge_face_colormap=cmap)

Some other parameters can be tweaked as well:

In [ ]:
cmap = plt.cm.Reds
edge_lc = "gray"
edge_lw = 4
node_fc = "black"
node_ec = "white"
node_lw = 2
node_size = 20

xgi.draw(
    H,
    pos,
    edge_lc=edge_lc,
    edge_lw=edge_lw,
    node_fc=node_fc,
    node_ec=node_ec,
    node_lw=node_lw,
    node_size=node_size,
    edge_face_colormap=cmap,
)

# Visualizing simplicial complexes

Simplicial complexes can be visualized using the same functions for node layout and drawing.

### Technical note
By definition, a simplicial complex object contains all sub-simplices. This would make the visualization heavy since all sub-simplices contained in a maximal simplex would overlap. The automatic solution for this, implemented by default in all the layouts, is to convert the simplicial complex into a hypergraph composed by solely by its maximal simplices.

### Visual note
To visually distinguish simplicial complexes from hypergraphs, the ```draw``` function will also show all links contained in each maximal simplices (while omitting simplices of intermediate orders). 

In [ ]:
SC = xgi.SimplicialComplex()
SC.add_simplices_from([[3, 4, 5], [3, 6], [6, 7, 8, 9], [1, 4, 10, 11, 12], [1, 4]])

In [ ]:
pos = xgi.pairwise_spring_layout(SC)

In [ ]:
xgi.draw(SC, pos)

# Example: generative model
We generate and visualize a [random hypergraph](https://doi.org/10.1093/comnet/cnx001).

In [ ]:
n = 100
is_connected = False
while not is_connected:
    H = xgi.random_hypergraph(n, [0.03, 0.0002, 0.00001])
    is_connected = xgi.is_connected(H)
pos = xgi.barycenter_spring_layout(H)

Since there are more nodes we reduce the ```node_size```

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.subplot(111)
xgi.draw(H, pos, node_size=10, ax=ax)

We can even size/color the nodes and edges by NodeStats or EdgeStats (e.g., degree, centrality, size, etc.)!

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.subplot(111)
xgi.draw(
    H,
    pos,
    node_size=H.nodes.degree,
    node_lw=H.nodes.degree,
    node_fc=H.nodes.degree,
    ax=ax,
)

### Degree
Using its simplest (higher-order) definition, the degree is the number of hyperedges (of any size) incident on a node.

In [ ]:
centers, heights = xgi.degree_histogram(H)

plt.figure(figsize=(12, 4))
ax = plt.subplot(111)

ax.bar(centers, heights)
ax.set_ylabel("Count")
ax.set_xlabel("Degree")
ax.set_xticks(np.arange(1, max(centers) + 1, step=1));